# McCall Model

Pablo Winant

### Job-Search Model

-   When unemployed in date, a job-seeker
    -   consumes unemployment benefit $c_t = \underline{c}$
    -   receives in every date $t$ a job offer $w_t$
        -   $w_t$ is i.i.d.,
        -   takes values $w_1, w_2, w_3$ with probabilities
            $p_1, p_2, p_3$
    -   if job-seeker accepts, becomes employed at rate $w_t$ in the
        next period
    -   else he stays unemployed
-   When employed at rate $w$
    -   worker consumes salary $c_t = w$
    -   with small probability $\lambda>0$ looses his job:
        -   starts next period unemployed
    -   otherwise stays employed at same rate
-   Objective: $\max E_0 \left\{ \sum \beta^t \log(w_t) \right\}$

**Discuss the Effects of the Parameters**

**What are the states, the controls, the reward of this problem ? Write
down the Bellman equation.**

**The states are :**
- Unemployed , offered $w=w_1$
- Unemployed , offered $w=w_2$
- Unemployed , offered $w=w_3$
- Employed , paid $w=w_1$
- Employed , paid $w=w_2$
- Employed , paid $w=w_3$

The state-space can be represented as $(employed,unemployed)\times (w_1,w_2,w_3)$

**The controls (action) are $accept/reject$ when Unemployed, the empty set when Employed**

**The reward is $U(\underline{c})$ when Unemployed, and $U(w_i)$ when Employed**

**Define a parameter structure for the model.**

In [1]:
m = (;
    wages = [0.9, 1.0, 1.1],
    probas = [1/3, 1/3, 1/3],
    β = 0.9,
    λ = 0.01,
    cbar = 0.8)
U(x) = log(x)

U (generic function with 1 method)

In [2]:
# inital guesses
V_U_0 = [0.0,0.0,0.0]
V_E_0 = [0.0,0.0,0.0]
x_0   = [false, false, false]

3-element Vector{Bool}:
 0
 0
 0

**Define a function
`value_update(V_U::Vector{Float64}, V_E::Vector{Float64}, x::Vector{Bool}, p::Parameters)::Tuple{Vector, Vector}`,
which takes in value functions tomorrow and a policy vector and return
updated values for today.**

**$\textrm{value\_update} : (V,x) \rightarrow T.V(x)$**

In [3]:
function value_update(V_U, V_E, x, m)
    (;wages, probas, cbar, β, λ) = m
    
    #! --- Employed
    # Compute the CONTINUATION value (partie E[...]) of being unemployed tomorrow 
    cont_V_U = sum( probas[i]*V_U[i] for i=1:3 )
    
    # new value of being employed today
    n_V_E = zeros(3)
    for i=1:3   # by state !
        n_V_E[i] = U(wages[i]) + β*(1-λ)*V_E[i] + β*λ*cont_V_U
    end      
    # or vector : n_V_E = U.(wages) + β*(1-λ)*V_E + β*λ*cont_V_U

    #! --- unemployed
    n_V_U = zeros(3)
    for i=1:3
        if x[i] # accept
            n_V_U[i] = U(cbar) + β*V_E[i]
        else
            n_V_U[i] = U(cbar) + β*cont_V_U
        end
    end
    
    return n_V_U, n_V_E 
end

value_update(V_U_0, V_E_0, x_0, m)

([-0.2231435513142097, -0.2231435513142097, -0.2231435513142097], [-0.10536051565782628, 0.0, 0.09531017980432493])

In [4]:
distance(v1::Vector,v2::Vector) = maximum(abs.(v2-v1))

distance (generic function with 1 method)

**Define a function
`policy_eval(x::Vector{Bool}, p::Parameter)::Tuple{Vector, Vector}`
which takes in a policy vector and returns the *value(s) of following
this policy forever*.**

**$\Longrightarrow$ On cherche donc $V^\star(x)$ pour un x donné.** 

**On utilise que : starting from any $V_0,\; \forall x:  \, T^nV_0(x) \rightarrow V^\star(x)$**

In [12]:
function policy_eval(x, m; T=100, τ_η = 10^(-10), verbose=false)
    V_U_0 = [0.0,0.0,0.0]
    V_E_0 = [0.0,0.0,0.0]

    for t=1:T
        V_U_1, V_E_1 = value_update(V_U_0,V_E_0,x,m) 
        η = distance( [V_U_0;V_E_0], [V_U_1;V_E_1] )

        V_U_0, V_E_0 = V_U_1, V_E_1

        if verbose
            println("Iteration $(t): η $(η)")
            println("Iteration $(t): V_U $(V_U_0)")
            println("Iteration $(t): V_E $(V_E_0)")
            println("---------------")
        end

        if η<τ_η
            return V_U_0, V_E_0
        end
    end

    error("No convergence")
end

policy_eval (generic function with 1 method)

In [13]:
policy_eval(x_0,m; T=300, verbose=true)

Iteration 1: η 0.2231435513142097
Iteration 1: V_U [-0.2231435513142097, -0.2231435513142097, -0.2231435513142097]
Iteration 1: V_E [-0.10536051565782628, 0.0, 0.09531017980432493]
---------------
Iteration 2: η 0.20082919618278872
Iteration 2: V_U [-0.4239727474969984, -0.4239727474969984, -0.4239727474969984]
Iteration 2: V_E [-0.2012450270707774, -0.0020082919618278875, 0.17822325804815056]
---------------
Iteration 3: η 0.18074627656450992
Iteration 3: V_U [-0.6047190240615083, -0.6047190240615083, -0.6047190240615083]
Iteration 3: V_E [-0.28848558950536196, -0.005605142865461633, 0.2502913479977541]
---------------
Iteration 4: η 0.16267164890805885
Iteration 4: V_U [-0.7673906729695672, -0.7673906729695672, -0.7673906729695672]
Iteration 4: V_E [-0.36784364712365736, -0.010436653509679891, 0.31287729965377026]
---------------
Iteration 5: η 0.14640448401725292
Iteration 5: V_U [-0.9137951569868201, -0.9137951569868201, -0.9137951569868201]
Iteration 5: V_E [-0.4400157213017311, -

([-2.2314355123054526, -2.2314355123054526, -2.2314355123054526], [-1.1508572035823978, -0.18424696823521042, 0.6901583510291687])

**Define a function
`bellman_step(V_E::Vector, V_U::Vector, p::Parameters)::Tuple{Vector, Vector, Vector}`
which returns updated values, together with improved policy rules.**

In [14]:
function bellman_step(V_U, V_E, m)
    (;wages, probas, cbar, β, λ) = m
    
    #! --- Employed (pareil que value_update car pas de CHOIX pour action)
    # Compute the CONTINUATION value (partie E[...]) of being unemployed tomorrow 
    cont_V_U = sum( probas[i]*V_U[i] for i=1:3 )

    # new value of being employed today (based on value function of tomorrow)
    n_V_E = zeros(3)
    for i=1:3   # by state !
        n_V_E[i] = U(wages[i]) + β*(1-λ)*V_E[i] + β*λ*cont_V_U
    end      
    # or vector : n_V_E = U.(wages) + β*(1-λ)*V_E + β*λ*cont_V_U

    #! --- unemployed
    n_V_U = zeros(3)
    #! update policy/decision rule : MAKE BEST CHOICE
    x = [false,false,false]
    for i=1:3
        # Pour chaque state : a(s) ∈ {accept,reject}
        v_accept = U(cbar) + β*V_E[i]
        v_reject = U(cbar) + β*cont_V_U
        if v_accept>v_reject
            # >> accept
            n_V_U[i] = v_accept
            x[i] = true
        else
            # >> reject
            n_V_U[i] = v_reject
            x[i] = false
        end
    end
    
    return n_V_U, n_V_E, x
end

bellman_step(V_U_0, V_E_0, m)

([-0.2231435513142097, -0.2231435513142097, -0.2231435513142097], [-0.10536051565782628, 0.0, 0.09531017980432493], Bool[0, 0, 0])

**Implement `Value Function`**

In [36]:
function value_function_iteration(m; T=100, τ_η = 10^(-10), verbose=false)

    V_U_0 = [0.0,0.0,0.0]
    V_E_0 = [0.0,0.0,0.0]

    for t=1:T
        V_U_1, V_E_1, x = bellman_step(V_U_0,V_E_0,m) 
        η = distance( [V_U_0;V_E_0], [V_U_1;V_E_1] )

        V_U_0, V_E_0 = V_U_1, V_E_1

        if verbose
            println("Iteration $(t): $(η) | $(x)")
        end

        if η<τ_η
            return V_U_0, V_E_0, x
        end
    end

    error("No convergence")
end

value_function_iteration(x,m; T=300, verbose=true)

Iteration 1: 0.2231435513142097 | Bool[0, 0, 0]
Iteration 2: 0.09588451141295112 | Bool[1, 1, 1]
Iteration 3: 0.08629606027165598 | Bool[1, 1, 1]
Iteration 4: 0.07618551515584138 | Bool[0, 1, 1]
Iteration 5: 0.06776620825824747 | Bool[0, 1, 1]
Iteration 6: 0.06017209711438726 | Bool[0, 1, 1]
Iteration 7: 0.05339623450323355 | Bool[0, 1, 1]
Iteration 8: 0.04737184303259734 | Bool[0, 1, 1]
Iteration 9: 0.042021967211819056 | Bool[0, 1, 1]
Iteration 10: 0.037273116454005106 | Bool[0, 1, 1]
Iteration 11: 0.033058519006658904 | Bool[0, 1, 1]
Iteration 12: 0.029318432293365393 | Bool[0, 1, 1]
Iteration 13: 0.025999667465517406 | Bool[0, 1, 1]
Iteration 14: 0.023054948225977645 | Bool[0, 1, 1]
Iteration 15: 0.020442277375651963 | Bool[0, 1, 1]
Iteration 16: 0.018124355344447007 | Bool[0, 1, 1]
Iteration 17: 0.01606805806256495 | Bool[0, 1, 1]
Iteration 18: 0.014243971400986988 | Bool[0, 1, 1]
Iteration 19: 0.01262597700763557 | Bool[0, 0, 1]
Iteration 20: 0.011258315445731748 | Bool[0, 0, 1]


([-0.12917371935610356, -0.12917371935610356, 0.5715802122005743], [-0.9579891504189799, 0.00862108482895072, 0.883026404003542], Bool[0, 0, 1])

From 19 to 196 make the same decision and just becomes more precise on the Value function

**Implement `Policy Iteration` and compare rates of convergence.**

In [37]:
function policy_function_iteration(m; T=100, τ_η = 10^(-10), verbose=false)

    V_U_0 = [0.0,0.0,0.0]
    V_E_0 = [0.0,0.0,0.0]
    x_0 = [false,false,false]

    for t=1:T

        V_U_0, V_E_0 = policy_eval(x_0, m; T=350)

        V_U_1, V_E_1, x = bellman_step(V_U_0,V_E_0,m) 
        
        η = maximum(abs.(x-x_0))
        x_0 = x

        if verbose
            println("Iteration $(t): $(η) | $(x)")
        end

        if η<τ_η
            return V_U_0, V_E_0, x
        end
    end

    error("No convergence")
end

policy_function_iteration(m; T=300, verbose=true)

Iteration 1: 1 | Bool[1, 1, 1]
Iteration 2: 1 | Bool[0, 1, 1]
Iteration 3: 1 | Bool[0, 0, 1]
Iteration 4: 0 | Bool[0, 0, 1]


([-0.12917371930468718, -0.12917371930468718, 0.5715802122519905], [-0.95798915039766, 0.008621084866072553, 0.8830264040549584], Bool[0, 0, 1])